In [1]:
import os,gc
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import cudf

In [2]:
df = cudf.read_parquet('../../data/train_data/train.parquet')
mx = df.ts.max()
df = df.loc[df.ts >= mx-60*60*24*7]

df['hour'] = df.ts % (60*60*24)
df['day'] = df.ts % (60*60*24*7)
print( df.shape )
df.head()

(55407186, 6)


,session,aid,ts,type,hour,day
86,0,723931,1660544235,0,22635,368235
87,0,1436439,1660544269,0,22669,368269
88,0,1693461,1660544287,0,22687,368287
89,0,1206554,1660544299,0,22699,368299
90,0,1110741,1660546139,0,24539,370139


In [3]:
df = df.sort_values(['session','ts'])
df['prev'] = df.groupby('session').aid.diff(1)
df['next'] = df.groupby('session').aid.diff(-1)

df['order'] = (df.type==2).astype('int8')
df['cart'] = (df.type==1).astype('int8')

In [4]:
df.head()

,session,aid,ts,type,hour,day,prev,next,order,cart
86,0,723931,1660544235,0,22635,368235,<NA>,-712508,0,0
87,0,1436439,1660544269,0,22669,368269,712508,-257022,0,0
88,0,1693461,1660544287,0,22687,368287,257022,486907,0,0
89,0,1206554,1660544299,0,22699,368299,-486907,95813,0,0
90,0,1110741,1660546139,0,24539,370139,-95813,764389,0,0


In [5]:
df['n'] = df.groupby(['session','aid','type']).aid.transform('count')
tmp = df.loc[df['type']==2].drop_duplicates(['session','aid']).groupby('aid').n.mean()
tmp2 = df.loc[df['type']==1].drop_duplicates(['session','aid']).groupby('aid').n.mean()

In [6]:
tmp.head()

aid
1611486     1.0
190414     <NA>
27160       1.0
441176     <NA>
686594     <NA>
Name: n, dtype: float64

In [7]:
tmp.max(), tmp2.max()

(122.0, 163.0)

In [8]:
item_features3 = df.groupby('aid').agg({'prev':'nunique','next':'nunique','order':'sum','cart':'sum'})
item_features3 = item_features3.merge(tmp,left_index=True, right_index=True, how='left')
item_features3.columns = ['prev3','next3','orders3','carts3','buy_count3']
item_features3 = item_features3.merge(tmp2,left_index=True, right_index=True, how='left')
item_features3.columns = ['prev3','next3','orders3','carts3','order_repeat3','cart_repeat3']
item_features3 = item_features3.fillna(-1)

i32 = ['prev3','next3','orders3','carts3']
for c in i32: item_features3[c] = item_features3[c].astype('int32')
    
f32 = ['order_repeat3','cart_repeat3']
for c in f32: item_features3[c] = item_features3[c].astype('float32')
    
del tmp, tmp2
gc.collect()

64

In [9]:
item_features3.head()

,prev3,next3,orders3,carts3,order_repeat3,cart_repeat3
aid,,,,,,
11477,11,10,2,2,-1.0,-1.0
11478,24,24,1,3,1.0,2.0
11473,2,2,0,1,-1.0,-1.0
11479,12,11,0,1,-1.0,2.0
11467,17,16,0,0,-1.0,-1.0


In [10]:
item_features = df.groupby('aid').agg({'type':'mean','aid':'count','session':'nunique','hour':'mean','day':'mean'})
item_features.columns = ['buy_ratio3','count_item3','count_user3','hour_mean3','day_mean3']

In [11]:
item_features.head()

,buy_ratio3,count_item3,count_user3,hour_mean3,day_mean3
aid,,,,,
0,0.000000,10,8,62784.800000,442944.800000
1,0.000000,4,4,59581.750000,534781.750000
2,0.000000,5,5,44185.800000,303385.800000
3,0.081633,1176,654,50518.395408,320738.803571
4,0.052632,57,37,58397.964912,272124.280702


In [12]:
item_features2 = df.groupby('aid').agg({'type':'std','hour':'std','day':'std'}).fillna(-1)
item_features2.columns = ['buy_ratio_std3','hour_std3','day_std3']

f32 = ['buy_ratio_std3','hour_std3','day_std3']
for c in f32: item_features2[c] = item_features2[c].astype('float32')

In [13]:
item_features2.head()

,buy_ratio_std3,hour_std3,day_std3
aid,,,
1653390,0.065795,20142.943359,182963.18750
1209028,0.262613,16947.570312,173690.62500
399895,0.754402,19166.580078,190340.78125
417981,0.000000,13594.806641,130647.31250
1371645,0.000000,22266.080078,158640.81250


In [14]:
item_features['repeat3'] = item_features.count_item3 / item_features.count_user3

f32 = ['buy_ratio3','hour_mean3','day_mean3','repeat3']
for c in f32: item_features[c] = item_features[c].astype('float32')

i32 = ['count_item3','count_user3']
for c in i32: item_features[c] = item_features[c].astype('int32')

In [15]:
item_features = cudf.concat([item_features,item_features2,item_features3],axis=1)

In [16]:
item_features.head()

,buy_ratio3,count_item3,count_user3,hour_mean3,day_mean3,repeat3,buy_ratio_std3,hour_std3,day_std3,prev3,next3,orders3,carts3,order_repeat3,cart_repeat3
aid,,,,,,,,,,,,,,,
0,0.000000,10,8,62784.800781,442944.81250,1.250000,0.000000,19575.779297,111253.203125,9,9,0,0,-1.0,-1.000000
1,0.000000,4,4,59581.750000,534781.75000,1.000000,0.000000,20764.863281,45779.808594,4,4,0,0,-1.0,-1.000000
2,0.000000,5,5,44185.800781,303385.81250,1.000000,0.000000,6137.582520,188237.781250,4,4,0,0,-1.0,-1.000000
3,0.081633,1176,654,50518.394531,320738.81250,1.798165,0.317119,16433.859375,166824.000000,161,181,15,66,1.0,1.538462
4,0.052632,57,37,58397.964844,272124.28125,1.540541,0.225282,16204.443359,160786.625000,42,37,0,3,-1.0,-1.000000


In [17]:
item_features.dtypes

buy_ratio3        float32
count_item3         int32
count_user3         int32
hour_mean3        float32
day_mean3         float32
repeat3           float32
buy_ratio_std3    float32
hour_std3         float32
day_std3          float32
prev3               int32
next3               int32
orders3             int32
carts3              int32
order_repeat3     float32
cart_repeat3      float32
dtype: object

In [18]:
COLS = [f'{x[:-1]}7' for x in item_features.columns]
item_features.columns = COLS
print(COLS)

['buy_ratio7', 'count_item7', 'count_user7', 'hour_mean7', 'day_mean7', 'repeat7', 'buy_ratio_std7', 'hour_std7', 'day_std7', 'prev7', 'next7', 'orders7', 'carts7', 'order_repeat7', 'cart_repeat7']


In [19]:
item_features.to_parquet('../../data/item_user_features/item20.pqt')